# Chapter 3: Keras and Data Retrieval in TensorFlow 2

<table align="left">
    <td>
        <a target="_blank" href="https://colab.research.google.com/github/thushv89/manning_tf2_in_action/blob/master/Ch03/3.2.Creating_Input_Pipelines.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
    </td>
</table>

## Importing necessary libraries and some setup

In [1]:
import os
import requests
from zipfile import ZipFile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.models import Sequential
import pandas as pd
import tensorflow_datasets as tfds

def fix_random_seed(seed):
    try:
        np.random.seed(seed)
    except NameError:
        print("Warning: Numpy is not imported. Setting the seed for Numpy failed.")
    try:
        tf.random.set_seed(seed)
    except NameError:
        print("Warning: TensorFlow is not imported. Setting the seed for TensorFlow failed.")
    try:
        random.seed(seed)
    except NameError:
        print("Warning: random module is not imported. Setting the seed for random failed.")

# Fixing the random seed
fix_random_seed(4321)
print("TensorFlow version: {}".format(tf.__version__))

data_dir = 'data'

if not os.path.exists(data_dir):
    os.makedirs(data_dir)


TensorFlow version: 2.1.0


## Using the `tf.data` API to retrieve data

Here we will be using the `tf.data` API to feed a dataset containing images of flowers. The dataset has a folder containing the images and a CSV file listing filenames and their corresponding label as an integer. We will write a TensorFlow data pipeline that does the following.

* Extract filenames and classes from the CSV
* Read in the images from the extracted filenames and resize them to 64x64
* Convert the class labels to one-hot encoded vectors
* Combine the processed images and one-hot encoded vectors to a single dataset
* Finally, shuffle the data and output as batches

### Downloading the data
The dataset is available at https://www.kaggle.com/olgabelitskaya/flower-color-images/data . 

You need to download the zip file available in this URL and place it in the `data` folder in the `Ch02` folder.

In [2]:
# Section 3.2

import os
from zipfile import ZipFile

# Extracting the flowers image data to a directory
if os.path.exists('data/flower-color-images.zip'):
    zfile = ZipFile('data/flower-color-images.zip')
    zfile.extractall('data')
else:
    print("Did you download the dataset as a zip file and place it in the Ch02/data folder?")

## Creating a tf.data.Dataset 

Here we are creating the `tf.data` pipeline that executes the above steps.

In [3]:
# Section 3.2
# Code listing 3.5

import tensorflow as tf
import os
import tensorflow.keras.backend as K

K.clear_session() # Making sure we are clearing out the TensorFlow graph

# Read the CSV file with TensorFlow
# The os.path.sep at the end is important for the get_image function
data_dir = os.path.join('data','flower_images', 'flower_images') + os.path.sep
assert os.path.exists(data_dir)
csv_ds = tf.data.experimental.CsvDataset(
    os.path.join(data_dir,'flower_labels.csv') , ("",-1), header=True
)
# Separate the image names and labels to two separate sets
fname_ds = csv_ds.map(lambda a,b: a)
label_ds = csv_ds.map(lambda a,b: b)

def get_image(file_path):
    
    img = tf.io.read_file(data_dir + file_path)
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_png(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img, [64, 64])

# Get the images by running get_image across all the filenames
image_ds = fname_ds.map(get_image)
print("The image dataset contains: {}".format(image_ds))
# Create onehot encoded labels from label data
label_ds = label_ds.map(lambda x: tf.one_hot(x, depth=10))
# Zip the images and labels together
data_ds = tf.data.Dataset.zip((image_ds, label_ds))

# Shuffle the data so that we get a mix of labels in every batch
data_ds = data_ds.shuffle(buffer_size= 20)
# Define a batch of size 5 
data_ds = data_ds.batch(5)
# Iterate through the data to see what it contains
for item in data_ds:
    print(item)
    break


The image dataset contains: <MapDataset shapes: (64, 64, 3), types: tf.float32>
(<tf.Tensor: shape=(5, 64, 64, 3), dtype=float32, numpy=
array([[[[0.5852941 , 0.5088236 , 0.39411768],
         [0.5852941 , 0.50980395, 0.4009804 ],
         [0.5862745 , 0.51176476, 0.40490198],
         ...,
         [0.82156867, 0.7294118 , 0.62352943],
         [0.82745105, 0.74509805, 0.6392157 ],
         [0.8284314 , 0.75098044, 0.64509803]],

        [[0.59607846, 0.51862746, 0.40980396],
         [0.59411764, 0.5235294 , 0.40882355],
         [0.59607846, 0.52254903, 0.41470593],
         ...,
         [0.82941186, 0.73921573, 0.63725495],
         [0.8313726 , 0.7411765 , 0.64215684],
         [0.82745105, 0.7401961 , 0.63823533]],

        [[0.60882354, 0.5294118 , 0.41960788],
         [0.6127451 , 0.532353  , 0.42352945],
         [0.6117647 , 0.53431374, 0.42549023],
         ...,
         [0.8343138 , 0.7441176 , 0.63823533],
         [0.82450986, 0.7303922 , 0.6303922 ],
         [0.812745

### Defining and training a model

Here we are defining a simple Convolution Neural Network (CNN) model to train it on the image data we just retrieved. You don't have to worry about the technical details of CNNs right now. We will discuss them in detail in the next chapter.

In [5]:
# Section 3.2

from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.models import Sequential

# Defining a Convolution neural network for you to train for the flowers data
# We will discuss convolution neural networks in more detail later
model = Sequential([
    Conv2D(64,(5,5), activation='relu', input_shape=(64,64,3)),
    Flatten(),
    Dense(10, activation='softmax')
])

# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

# Training the model with the tf.data pipeline
model.fit(data_ds, epochs=10)

Epoch 1/10
42/42 [==============================] - 1s 24ms/step - loss: 3.1604 - acc: 0.2571
Epoch 2/10
42/42 [==============================] - 1s 14ms/step - loss: 1.4359 - acc: 0.5190
Epoch 3/10
42/42 [==============================] - 1s 14ms/step - loss: 0.5586 - acc: 0.8286
Epoch 4/10
42/42 [==============================] - 1s 14ms/step - loss: 0.2442 - acc: 0.9333
Epoch 5/10
42/42 [==============================] - 1s 14ms/step - loss: 0.1048 - acc: 0.9762
Epoch 6/10
42/42 [==============================] - 1s 14ms/step - loss: 0.0399 - acc: 0.9952
Epoch 7/10
42/42 [==============================] - 1s 14ms/step - loss: 0.0678 - acc: 0.9810
Epoch 8/10
42/42 [==============================] - 1s 14ms/step - loss: 0.0577 - acc: 0.9810
Epoch 9/10
42/42 [==============================] - 1s 14ms/step - loss: 0.0126 - acc: 1.0000
Epoch 10/10
42/42 [==============================] - 1s 15ms/step - loss: 0.0019 - acc: 1.0000


## Using Keras data generators to retrieve data

Instead of `tf.data` API let us use the Keras `ImageDataGenerator` to retrieve the data. As you can see, the `ImageDataGenerator` involves much less code than the using the `tf.data` API. 

In [6]:
# Section 3.2
# Code listing 3.6

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import pandas as pd

data_dir = os.path.join('data','flower_images', 'flower_images')

# Defining an image data generator provided in Keras
img_gen = ImageDataGenerator()

# Reading the CSV files containing filenames and labels
labels_df = pd.read_csv(os.path.join(data_dir, 'flower_labels.csv'), header=0)

# Generating data using the flow_from_dataframe function
gen_iter = img_gen.flow_from_dataframe(
    dataframe=labels_df, directory=data_dir, x_col='file', y_col='label', class_mode='raw', batch_size=2, target_size=(64,64))

# Iterating through the data
for item in gen_iter:
    print(item)
    break

Found 210 validated image filenames.
(array([[[[ 10.,  11.,  11.],
         [ 51.,  74.,  46.],
         [ 36.,  56.,  32.],
         ...,
         [  4.,   4.,   3.],
         [ 16.,  25.,  11.],
         [ 17.,  18.,  13.]],

        [[ 12.,  13.,  10.],
         [ 22.,  29.,  18.],
         [ 35.,  46.,  26.],
         ...,
         [  3.,   3.,   3.],
         [  6.,   6.,   5.],
         [  9.,  10.,   9.]],

        [[ 11.,  11.,  10.],
         [ 13.,  14.,  12.],
         [ 45.,  60.,  32.],
         ...,
         [  4.,   4.,   4.],
         [  8.,   8.,   8.],
         [ 13.,  15.,  12.]],

        ...,

        [[ 63.,  98.,  51.],
         [ 60.,  89.,  46.],
         [ 37.,  59.,  31.],
         ...,
         [ 58.,  88.,  46.],
         [ 55.,  83.,  43.],
         [ 52.,  78.,  39.]],

        [[ 64.,  96.,  46.],
         [ 54.,  92.,  40.],
         [ 68.,  96.,  48.],
         ...,
         [ 48.,  77.,  39.],
         [ 49.,  82.,  39.],
         [ 48.,  78.,  39.]],

## Using the `tensorflow-datasets` library

Here we will use the `tensorflow-datasets` package. It is a curated list of popular datasets available for machine learning projects. With this package you can download a dataset in a single line. This means you don't have to worry about downloading/extracting/formatting data manually. All of that will be already done when you import data using the `tensorflow-datasets` library.

### Lists the available datasets

In [7]:
# Section 3.2

import tensorflow_datasets as tfds
import tensorflow as tf
# See all registered datasets
tfds.list_builders()

['abstract_reasoning',
 'aeslc',
 'aflw2k3d',
 'amazon_us_reviews',
 'arc',
 'bair_robot_pushing_small',
 'beans',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'cos_e',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'deep_weeds',
 'definite_pronoun_resolution',
 'diabetic_retinopathy_detection',
 'div2k',
 'dmlab',
 'downsampled_imagenet',
 'dsprites',
 'dtd',
 'duke_ultrasound',
 'dummy_dataset_shared_generator',
 'dummy_mnist',
 'emnist',
 'eraser_multi_rc',
 'esnli',
 'eurosat',
 'fashion_mnist',
 'flic',
 'flores',
 'food101',
 'gap',
 'gigaword',
 'glue',
 'gr

### Download the Cifar10 dataset and view information

In [8]:
# Section 3.2

import tensorflow_datasets as tfds
import tensorflow.keras.backend as K

K.clear_session() # Making sure we are clearing out the TensorFlow graph

# Load a given dataset by name, along with the DatasetInfo
data, info = tfds.load("cifar10", with_info=True)
print(info)

tfds.core.DatasetInfo(
    name='cifar10',
    version=3.0.0,
    description='The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.',
    homepage='https://www.cs.toronto.edu/~kriz/cifar.html',
    features=FeaturesDict({
        'image': Image(shape=(32, 32, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=60000,
    splits={
        'test': 10000,
        'train': 50000,
    },
    supervised_keys=('image', 'label'),
    citation="""@TECHREPORT{Krizhevsky09learningmultiple,
        author = {Alex Krizhevsky},
        title = {Learning multiple layers of features from tiny images},
        institution = {},
        year = {2009}
    }""",
    redistribution_info=,
)



### Exploring the data 

Here we will print the `data` and see what it provides. Then we will need to batch the data as data is provided as individual samples when you import it from `tensorflow-datasets`.

In [9]:
print(data)

{'test': <DatasetV1Adapter shapes: {image: (32, 32, 3), label: ()}, types: {image: tf.uint8, label: tf.int64}>, 'train': <DatasetV1Adapter shapes: {image: (32, 32, 3), label: ()}, types: {image: tf.uint8, label: tf.int64}>}


In [10]:
# Section 3.2

import tensorflow as tf

# Defining a dataset with batch size 16
train_ds = data["train"].batch(16)

# Creating a dataset that returns an (image, one-hot label) tuple
def format_data(x):
    return (x["image"], tf.one_hot(x["label"], depth=10))
train_ds = train_ds.map(format_data)

# Iterating the dataset
for item in train_ds:
    print(item)
    break

(<tf.Tensor: shape=(16, 32, 32, 3), dtype=uint8, numpy=
array([[[[143,  96,  70],
         [141,  96,  72],
         [135,  93,  72],
         ...,
         [ 96,  37,  19],
         [105,  42,  18],
         [104,  38,  20]],

        [[128,  98,  92],
         [146, 118, 112],
         [170, 145, 138],
         ...,
         [108,  45,  26],
         [112,  44,  24],
         [112,  41,  22]],

        [[ 93,  69,  75],
         [118,  96, 101],
         [179, 160, 162],
         ...,
         [128,  68,  47],
         [125,  61,  42],
         [122,  59,  39]],

        ...,

        [[187, 150, 123],
         [184, 148, 123],
         [179, 142, 121],
         ...,
         [198, 163, 132],
         [201, 166, 135],
         [207, 174, 143]],

        [[187, 150, 117],
         [181, 143, 115],
         [175, 136, 113],
         ...,
         [201, 164, 132],
         [205, 168, 135],
         [207, 171, 139]],

        [[195, 161, 126],
         [187, 153, 123],
         [186, 151

### Training a simple CNN on the Cifar10 data

In [11]:
# Section 3.2

from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.models import Sequential

# Defining a simple convolution neural network to process the CIFAR data
model = Sequential([
    Conv2D(64,(5,5), activation='relu', input_shape=(32,32,3)),
    Flatten(),
    Dense(10, activation='softmax')
])
# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

# Fitting the model on the data for 25 epochs
model.fit(train_ds, epochs=25)

Epoch 1/25
3125/3125 [==============================] - 37s 12ms/step - loss: 3.1641 - acc: 0.1257
Epoch 2/25
3125/3125 [==============================] - 22s 7ms/step - loss: 2.2790 - acc: 0.1293
Epoch 3/25
3125/3125 [==============================] - 23s 7ms/step - loss: 2.2457 - acc: 0.1509
Epoch 4/25
3125/3125 [==============================] - 25s 8ms/step - loss: 2.2140 - acc: 0.1586
Epoch 5/25
3125/3125 [==============================] - 25s 8ms/step - loss: 2.1870 - acc: 0.1669
Epoch 6/25
3125/3125 [==============================] - 22s 7ms/step - loss: 2.1570 - acc: 0.1856
Epoch 7/25
3125/3125 [==============================] - 21s 7ms/step - loss: 2.1155 - acc: 0.2001
Epoch 8/25
3125/3125 [==============================] - 21s 7ms/step - loss: 2.0815 - acc: 0.2108
Epoch 9/25
3125/3125 [==============================] - 24s 8ms/step - loss: 2.0614 - acc: 0.2199
Epoch 10/25
3125/3125 [==============================] - 23s 7ms/step - loss: 2.0388 - acc: 0.2287
Epoch 11/25
3125/3